In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

Variables

In [305]:
print_to_excel = True

sheet_name = 'FX Exposure_Balance Sheet'
month = ['nov','dec']
month = month[0]

In [ ]:
folders = [x for x in os.listdir() if month in x]
files_list = []
folders_for_df = []
for count in range(len(folders)):
    files_folder_list = os.listdir(f'{folders[count]}')
    files_folder_list = [x for x in files_folder_list if '~' not in x]
    files_list += files_folder_list
    folders_for_df += [folders[count] for x in range(len(files_folder_list))]
files_list

['BRFTO FX Exposure Model 28.11.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 28.11.2024 - With Pnl & Tax Credit.xlsx',
 'Cópia de BRFHRG FX Exposure 28.11.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFHRG FX Exposure 29.11.2024 - With pnl  Tax Credit - New model.xlsx',
 'BRFTO FX Exposure Model 29.11.2024 - With Pnl & Tax Credit -New Model.xlsx',
 'BRSFT FX Exposure Model 29.11.2024 - With Pnl & Tax Credit.xlsx']

In [287]:
files_df = pd.DataFrame({'files':files_list, 'folder':folders_for_df})
files_df['day'] = files_df.files.str.extract(r'([0-9]{2}.[0-9]{2}.[0-9]{4})')
files_df['company'] = files_df.files.str.extract(r'(BR\w* )')
files_df

,files,folder,day,company
0,BRFTO FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRFTO
1,BRSFT FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRSFT
2,Cópia de BRFHRG FX Exposure 28.11.2024 - With ...,28 nov,28.11.2024,BRFHRG
3,BRFHRG FX Exposure 29.11.2024 - With pnl Tax ...,29 nov,29.11.2024,BRFHRG
4,BRFTO FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024,BRFTO
5,BRSFT FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024,BRSFT


In [292]:
files_df_merge = files_df.merge(files_df, on='company', suffixes=['_old','_new'])
files_df_couple = files_df_merge[(files_df_merge.day_old!=files_df_merge.day_new)&(files_df_merge.day_old<files_df_merge.day_new)]
files_df_couple = files_df_couple.reset_index(drop=True)
files_df_couple

,files_old,folder_old,day_old,company,files_new,folder_new,day_new
0,BRFTO FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRFTO,BRFTO FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024
1,BRSFT FX Exposure Model 28.11.2024 - With Pnl ...,28 nov,28.11.2024,BRSFT,BRSFT FX Exposure Model 29.11.2024 - With Pnl ...,29 nov,29.11.2024
2,Cópia de BRFHRG FX Exposure 28.11.2024 - With ...,28 nov,28.11.2024,BRFHRG,BRFHRG FX Exposure 29.11.2024 - With pnl Tax ...,29 nov,29.11.2024


Counting

In [358]:
final_df = pd.DataFrame()
for string_id in files_df_couple.index:
    try:
        # read old data
        old_path = f'{files_df_couple.loc[string_id, "folder_old"]}\\{files_df_couple.loc[string_id, "files_old"]}'
        old_data = pd.read_excel(old_path, sheet_name=sheet_name)
        old_data_date = files_df_couple.loc[string_id, "day_old"]
        print(old_path)

        # read new data
        new_path = f'{files_df_couple.loc[string_id, "folder_new"]}\\{files_df_couple.loc[string_id, "files_new"]}'
        new_data = pd.read_excel(new_path, sheet_name=sheet_name)
        new_data_date = files_df_couple.loc[string_id, "day_new"]
        print(new_path)

        company_name = files_df_couple.loc[string_id, "company"]

        count_data_date = 0
        for data_date in [old_data, new_data]:
            
            count_data_date += 1

            list_to_del = [np.nan, '0', 0]
            # del empty rows
            data_date = data_date[(~data_date.isin(list_to_del)).any(axis=1)]
            # del empty cols
            cols_list = (~data_date.isin(list_to_del)).any(axis=0)
            cols_list = cols_list[cols_list==True].index.tolist()
            data_date = data_date[cols_list]

            # find string with will be a header
            id_header_row = data_date[data_date.apply(lambda row: row.astype(str).str.lower().str.contains('usd denominated').any(), axis=1)].index.item()
            data_date = data_date.loc[id_header_row:, :]

            # find col with wiil cut df in two parts
            cols_list = data_date.apply(lambda row: row.astype(str).str.lower().str.contains('current liabilities').any(), axis=0)
            cut_col = cols_list[cols_list==True].index.item()

            asserts_part = data_date.loc[:, :cut_col]
            liabilities_part = data_date.loc[:, cut_col:]

            count = 0
            for data in [asserts_part, liabilities_part]:
                count += 1
                cols_list = ['deals'] + list(map(str, data.loc[id_header_row, :].tolist()))[1:]
                data.columns = cols_list
                cols_list = [x for x in cols_list if x != 'nan']

                data = data[~data.deals.isna()]
                id_tail_row = data[data.apply(lambda row: row.astype(str).str.lower().str.contains('total').any(), axis=1)].index.tolist()[0]
                data = data.loc[:id_tail_row,:]

                if count == 1:
                    asserts_part = data[cols_list]
                if count == 2:
                    liabilities_part = data[cols_list]

            # concat parts 
            concat_data = pd.concat([asserts_part,liabilities_part], axis = 0, ignore_index=True)
            concat_data.iloc[:,1:] = concat_data.iloc[:,1:].fillna(0).apply(pd.to_numeric)
            # deal type col
            deal_type_list = ['current assets', 'non-current assets', 'current liabilities', 'non-current liabilities']	
            concat_data['deal_type'] = concat_data.deals
            concat_data.loc[~concat_data.deal_type.str.lower().isin(deal_type_list), 'deal_type'] = np.nan
            concat_data.deal_type = concat_data.deal_type.fillna(method='ffill')
            # melt data
            concat_data_melt = concat_data.melt(value_vars=concat_data.iloc[:,1:].columns.tolist(), value_name='value', id_vars=['deal_type','deals'], var_name='type')

            if count_data_date == 1:
                    old_data_melt = concat_data_melt
                    old_data_melt['day'] = old_data_date
            if count_data_date == 2:
                    new_data_melt = concat_data_melt
                    new_data_melt['day'] = new_data_date

        # concat old and new data
        old_data_melt['merge_col'] = old_data_melt['deal_type'].str.strip().str.lower() +old_data_melt['deals'].str.strip().str.lower() + old_data_melt['type'].str.strip().str.lower()
        new_data_melt['merge_col'] = new_data_melt['deal_type'].str.strip().str.lower() +new_data_melt['deals'].str.strip().str.lower() + new_data_melt['type'].str.strip().str.lower()
        old_data_melt = old_data_melt.fillna(0)[old_data_melt.value!=0]
        new_data_melt = new_data_melt.fillna(0)[new_data_melt.value!=0]

        merge_old_new_data = old_data_melt.merge(new_data_melt, on='merge_col', how='outer', suffixes=['_old', '_new']).fillna(0)
        # data without nulls
        merge_old_new_data = merge_old_new_data[(merge_old_new_data.value_old!=0)|(merge_old_new_data.value_new!=0)]
        # count the difference
        merge_old_new_data['diff'] = abs(merge_old_new_data.value_old.abs() - merge_old_new_data.value_new.abs())
        merge_old_new_data = merge_old_new_data.sort_values('diff', ascending=False)
        merge_old_new_data['pct_diff'] =  merge_old_new_data['diff'] / abs(merge_old_new_data[['value_old','value_new']]).max(axis=1)
        # final strokes
        merge_old_new_data['company'] = company_name
        merge_old_new_data = merge_old_new_data[['company','deal_type_old','deals_old','type_old','day_old','day_new','value_old','value_new','diff','pct_diff']]

        final_df = pd.concat([final_df,merge_old_new_data])
    except Exception as e:
        print(e, 'in string ',string_id)

28 nov\BRFTO FX Exposure Model 28.11.2024 - With Pnl & Tax Credit -New Model.xlsx
29 nov\BRFTO FX Exposure Model 29.11.2024 - With Pnl & Tax Credit -New Model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  concat_data.iloc[:,1:] = concat_data.iloc[:,1:].fillna(0).apply(pd.to_numeric)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concat_data.deal_type = concat_data.deal_type.fillna(method='ffill')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

28 nov\BRSFT FX Exposure Model 28.11.2024 - With Pnl & Tax Credit.xlsx
29 nov\BRSFT FX Exposure Model 29.11.2024 - With Pnl & Tax Credit.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  concat_data.iloc[:,1:] = concat_data.iloc[:,1:].fillna(0).apply(pd.to_numeric)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concat_data.deal_type = concat_data.deal_type.fillna(method='ffill')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

28 nov\Cópia de BRFHRG FX Exposure 28.11.2024 - With pnl  Tax Credit - New model.xlsx
29 nov\BRFHRG FX Exposure 29.11.2024 - With pnl  Tax Credit - New model.xlsx


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  concat_data.iloc[:,1:] = concat_data.iloc[:,1:].fillna(0).apply(pd.to_numeric)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:65: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concat_data.deal_type = concat_data.deal_type.fillna(method='ffill')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18252\1828978621.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

In [304]:
final_df.company.unique()

array(['BRFTO ', 'BRSFT ', 'BRFHRG '], dtype=object)

To excel

In [359]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_Brazil_monitoring.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        files_df_couple.to_excel(writer, sheet_name=f'{month}_files', index=False)
        final_df.to_excel(writer, sheet_name=f'{month}_data', index=False)